In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df2 = pd.read_csv("weather_data/cities.csv")
city_data_df2.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Nanortalik,GL,2022-07-20 22:56:54,60.1432,-45.2371,49.93,86,23,13.67
1,1,Namibe,AO,2022-07-20 22:54:20,-15.1961,12.1522,62.56,80,1,5.35
2,2,Waingapu,ID,2022-07-20 22:52:56,-9.6567,120.2641,72.45,77,33,7.76
3,3,Kavaratti,IN,2022-07-20 22:56:55,10.5669,72.6420,81.05,79,100,19.89
4,4,Arraial Do Cabo,BR,2022-07-20 22:54:39,-22.9661,-42.0278,71.46,83,40,5.75


In [3]:
# Get the data types.
city_data_df2.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of temperature

# Get the latitude and longitude.
locations = city_data_df2[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df2["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
    
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df2[["Lat", "Lng"]]
humidity = city_data_df2["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent cloudiness
locations = city_data_df2[["Lat", "Lng"]]
clouds = city_data_df2["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of wind speed
locations = city_data_df2[["Lat", "Lng"]]
wind = city_data_df2["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df2.loc[(city_data_df2["Max Temp"] <= max_temp) & \
                                        (city_data_df2["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Kavaratti,IN,2022-07-20 22:56:55,10.5669,72.6420,81.05,79,100,19.89
6,6,Georgetown,MY,2022-07-20 22:56:07,5.4112,100.3354,80.89,90,20,3.44
8,8,Atlantic City,US,2022-07-20 22:56:56,39.3643,-74.4229,87.03,70,1,14.03
15,15,Kapaa,US,2022-07-20 22:54:50,22.0752,-159.3190,81.48,74,11,7.00
19,19,Jamestown,US,2022-07-20 22:56:59,42.0970,-79.2353,85.01,56,27,7.00


In [11]:
preferred_cities_df.count()

City_ID       174
City          174
Country       174
Date          174
Lat           174
Lng           174
Max Temp      174
Humidity      174
Cloudiness    174
Wind Speed    174
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Kavaratti,IN,81.05,10.5669,72.6420,
6,Georgetown,MY,80.89,5.4112,100.3354,
8,Atlantic City,US,87.03,39.3643,-74.4229,
15,Kapaa,US,81.48,22.0752,-159.3190,
19,Jamestown,US,85.01,42.0970,-79.2353,


In [16]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Kavaratti,IN,81.05,10.5669,72.6420,Green leaf
6,Georgetown,MY,80.89,5.4112,100.3354,Cititel Penang
8,Atlantic City,US,87.03,39.3643,-74.4229,Courtyard by Marriott Atlantic City Beach Block
15,Kapaa,US,81.48,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
19,Jamestown,US,85.01,42.0970,-79.2353,Hampton Inn & Suites Jamestown


In [19]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [23]:
# Store the DataFrame row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_info

['\n<dl>\n<dt>Hotel Name</dt><dd>Green leaf</dd>\n<dt>City</dt><dd>Kavaratti</dd>\n<dt>Country</dt><dd>IN</dd>\n<dt>Max Temp</dt><dd>81.05 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Cititel Penang</dd>\n<dt>City</dt><dd>Georgetown</dd>\n<dt>Country</dt><dd>MY</dd>\n<dt>Max Temp</dt><dd>80.89 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Courtyard by Marriott Atlantic City Beach Block</dd>\n<dt>City</dt><dd>Atlantic City</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Max Temp</dt><dd>87.03 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Sheraton Kauai Coconut Beach Resort</dd>\n<dt>City</dt><dd>Kapaa</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Max Temp</dt><dd>81.48 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hampton Inn & Suites Jamestown</dd>\n<dt>City</dt><dd>Jamestown</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Max Temp</dt><dd>85.01 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Muesse</dd>\n<dt>City</dt><dd>Brazzaville</dd>\n<dt>Country</dt><dd>CG</dd>\n<dt>Max

In [24]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))